In [1]:
# !pip install SimPEG

In [2]:
import numpy as np
import scipy.sparse as sp
from SimPEG import Mesh, Utils, Solver
from scipy.constants import mu_0, epsilon_0
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Sensitivity computuation for 1D magnetotelluric problem

##  Purpose

XX

- XXX

## Discretizations: 

$$ \mathbf{Grad} \ \mathbf{E}_x + \imath \omega \mathbf{M}^{f2cc}_{\mu} \ \mathbf{H}_y  = - \mathbf{B}\mathbf{E}_x^{bc}$$

$$ \mathbf{M}^{cc}_{\hat{\sigma}} \ \mathbf{E}_x + \mathbf{Div} \ \mathbf{H}_y  = \boldsymbol{0}$$


- $\mathbf{E}_x$: Discrete $E_x$ component $[nC \times 1]$

- $\mathbf{H}_y$: Dicrete $H_x$ component $[(nC+1) \times 1]$

- $ \mathbf{Grad}$: Discrete gradient operator $[nC \times (nC+1)]$

- $ \mathbf{Div}$: Discrete divergence operator $[(nC+1) \times nC]$

- $\mathbf{M}^{f2cc}_{\mu}$: $\mathbf{diag}(\mathbf{Av}^T  \boldsymbol{\mu})$ $[(nC+1) \times (nC+1)]$

- $\mathbf{M}^{cc}_{\hat{\sigma}}$: $\mathbf{diag}(\boldsymbol{\hat{\sigma}})$ $[nC \times nC]$

Above discrete Maxwell's equations can be expressed as 

$$\mathbf{A}\mathbf{u} = \mathbf{rhs}$$

where 

- $\mathbf{A} =  \begin{bmatrix}
       \mathbf{Grad} & \imath \omega \mathbf{M}^{f2cc}_{\mu} \\[0.3em]
       \mathbf{M}^{cc}_{\hat{\sigma}} & \mathbf{Div}           \\[0.3em]
     \end{bmatrix}$


- $\mathbf{u} = \begin{bmatrix}
       \mathbf{E}_x \\[0.3em]
       \mathbf{H}_y \\[0.3em]
    \end{bmatrix}$


- $\mathbf{rhs} = \begin{bmatrix}
       - \mathbf{B}\mathbf{E}_x^{bc} \\[0.3em]
       \boldsymbol{0} \\[0.3em]
    \end{bmatrix}$


## What is data?

Measured data in general can be defined as:

$$ \mathbf{d} = P(\mathbf{u}) $$

where $P(\cdot)$ is a evaluation functional. For instance if we set complex impedance as our data then

$$ \mathbf{d} = \mathbf{Z}_{xy} = - \mathbf{P}_{0}(\mathbf{E_x} / \mathbf{H_y}) $$

Often real and imagniary part of the impedance considered as a separate data in this case:

$$\mathbf{d} =  \begin{bmatrix}
       Re[\mathbf{Z_{xy}}]  \\[0.3em]
       Im[\mathbf{Z_{xy}}]  \\[0.3em]
     \end{bmatrix}$$

Or we could choose apparent resistivity and phase of the impedance:

$$\mathbf{d} =  \begin{bmatrix}
       \rho_a \\[0.3em]
       \phi  \\[0.3em]
     \end{bmatrix}$$

where 

$$ \rho_a = \frac{|\mathbf{Z}_{xy}^2|}{\mu_0\omega} $$

$$ \phi = tan^{-1}(\frac{Im[\mathbf{Z_{xy}}]}{Re[\mathbf{Z_{xy}}]}) $$

## Sensitivity of datum with regard to $\sigma$:

Sensitivity function indicates changes in datum due to perturbed conductivity, and hence it can be defined as 

$$ J = \frac{d P(u)}{d \sigma}$$

## Computing Jvec

In [3]:
# This will be inputs
frequency = np.logspace(-3, 2, 25)
fmax, fmin = frequency.max(), frequency.min()
max_depth_core = 5000.
rho_half = 100.

In [4]:
print ("Smallest cell size = %d m") % (500*np.sqrt(rho_half/fmax) / 4.)
print ("Padding distance = %d m") % (500*np.sqrt(rho_half/fmin) * 2)
cs = 500*np.sqrt(rho_half/fmax) / 10.
skindepth2 = 500*np.sqrt(100/fmin) * 2

npad = 1
blength = cs*1.3**(np.arange(npad)+1)
while blength < skindepth2:
    npad+=1
    blength = (cs*1.3**(np.arange(npad)+1)).sum()

ncz = int(max_depth_core / cs)
hz = [(cs, npad, -1.3), (cs, ncz)]
mesh = Mesh.TensorMesh([hz], x0='N')
sigma = np.ones(mesh.nC) * 1./rho_half

Smallest cell size = 125 m
Padding distance = 316227 m


In [5]:
def dpred(sigma, dtype="ri"):
    f = 100.
    mu = np.ones(mesh.nC)*mu_0 # magnetic permeability values for all cells
    epsilon = np.ones(mesh.nC)*epsilon_0 # dielectric constant values for all cells
    omega = 2*np.pi*f # Angular frequency (rad/s)
    sigmahat = sigma # Assume sigmahat = sigma
    # In reality ...
    #         sigmahat = sigma + 1j*epsilon*omega # sigmahat = sigma + 1j*omega*epsilon
    Div = mesh.faceDiv # Divergence matrix
    mesh.setCellGradBC([['dirichlet', 'dirichlet']]) # Setup boundary conditions
    Grad = mesh.cellGrad # Gradient matrix
    B = mesh.cellGradBC  # a matrix for boundary conditions
    Exbc = np.r_[0., 1.] # boundary values for Ex
    Msighat = Utils.sdiag(sigmahat) 
    Mmu = Utils.sdiag(mesh.aveF2CC.T * mu) 

    tempUp = sp.hstack((Grad, 1j*omega*Mmu)) # Top row of A matrix
    tempDw = sp.hstack((Msighat, Div)) # Bottom row of A matrix
    A = sp.vstack((tempUp, tempDw)) # Full A matrix
    rhs = np.r_[-B*Exbc, np.zeros(mesh.nC)] # Right-hand side   

    Ainv = Solver(A) # Factorize A matrix
    u = Ainv*rhs   # Solve A^-1 rhs = u
    Ex = u[:mesh.nC] # Extract Ex from uution vector u
    Hy = u[mesh.nC:mesh.nC+mesh.nN] # Extract Hy from solution vector u    
    P0 = sp.coo_matrix(
        (np.r_[1.], (np.r_[0], np.r_[len(u)-1])), shape=(1, len(u))
             )
    P0 = P0.tocsr()
    Zxy = - 1./(P0*u)
    if dtype == "ri":
        return np.r_[Zxy.real, Zxy.imag]
    elif dtype == "rhopha":
        rhoa = abs(Zxy)**2 / (mu_0*omega)
        phase = np.rad2deg(np.arctan(Zxy.imag / Zxy.real))
        return np.r_[rhoa, phase]

def Jvec(sigma, v, dtype="ri"):
    f = 100.
    mu = np.ones(mesh.nC)*mu_0 # magnetic permeability values for all cells
    epsilon = np.ones(mesh.nC)*epsilon_0 # dielectric constant values for all cells
    omega = 2*np.pi*f # Angular frequency (rad/s)
    sigmahat = sigma # Assume sigmahat = sigma
    # In reality ...
    #         sigmahat = sigma + 1j*epsilon*omega # sigmahat = sigma + 1j*omega*epsilon
    Div = mesh.faceDiv # Divergence matrix
    mesh.setCellGradBC([['dirichlet', 'dirichlet']]) # Setup boundary conditions
    Grad = mesh.cellGrad # Gradient matrix
    B = mesh.cellGradBC  # a matrix for boundary conditions
    Exbc = np.r_[0., 1.] # boundary values for Ex
    Msighat = Utils.sdiag(sigmahat) 
    Mmu = Utils.sdiag(mesh.aveF2CC.T * mu) 

    tempUp = sp.hstack((Grad, 1j*omega*Mmu)) # Top row of A matrix
    tempDw = sp.hstack((Msighat, Div)) # Bottom row of A matrix
    A = sp.vstack((tempUp, tempDw)) # Full A matrix
    rhs = np.r_[-B*Exbc, np.zeros(mesh.nC)] # Right-hand side   

    Ainv = Solver(A) # Factorize A matrix
    u = Ainv*rhs   # Solve A^-1 rhs = u
    Ex = u[:mesh.nC] # Extract Ex from uution vector u
    Hy = u[mesh.nC:mesh.nC+mesh.nN] # Extract Hy from solution vector u    
    P0 = sp.coo_matrix(
        (np.r_[1.], (np.r_[0], np.r_[len(u)-1])), shape=(1, len(u))
             )
    P0 = P0.tocsr()
    Zxy = - 1./(P0*u)

    dAdsig_u_v = np.r_[np.zeros_like(Hy), Utils.sdiag(Ex)*v]
    dudsig_v = - (Ainv * (dAdsig_u_v))
    dZdsig_v = P0 * (Utils.sdiag(1./(u**2)) * dudsig_v)
    if dtype == "ri":
        dZrdsig_v = dZdsig_v.real
        dZidsig_v = dZdsig_v.imag
        return np.r_[dZrdsig_v, dZidsig_v]
    elif dtype == "rhopha":
        dZadZ = Zxy.conjugate() / abs(Zxy)
        drhodZa = 2. * abs(Zxy) / (mu_0*omega)
        drhodZ = drhodZa * dZadZ
        drhodsig_v = (drhodZ * dZdsig_v).real
#         dphadZ = 0.
#         dphadsig_v = (dphadZ * dZdsig_v).real
        return np.r_[drhodsig_v, 0.]

### Order test: Jvec

In [30]:
from SimPEG import Tests
def derChk(m):
    return [dpred(m, dtype="ri"), lambda mx: Jvec(m, mx, dtype="ri")]
Tests.checkDerivative(derChk, sigma, plotIt=False, num=3, eps=1e-20, dx=sigma*3)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.455e-02     7.603e-03      nan
 1   1.00e-02    4.122e-03     9.253e-05      1.915
 2   1.00e-03    4.205e-04     9.460e-07      1.990
========================= PASS! =========================
Go Test Go!



True

In [31]:
from SimPEG import Tests
def derChk(m):
    return [dpred(m, dtype="rhopha"), lambda mx: Jvec(m, mx, dtype="rhopha")]
Tests.checkDerivative(derChk, sigma, plotIt=False, num=3, eps=1e-20, dx=sigma*3)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    2.247e+01     6.791e+00      nan
 1   1.00e-02    2.840e+00     8.759e-02      1.889
 2   1.00e-03    2.917e-01     2.005e-03      1.640
========================= PASS! =========================
Once upon a time, a happy little test passed.



True

## Computing Jtvec

In [19]:
def misfit(sigma, dtype="ri", dobs=None):
    r = dpred(sigma, dtype=dtype) - dobs
    return 0.5 * np.linalg.norm(r)**2

def Jtvec(sigma, v, dtype="ri"):
    f = 100.
    mu = np.ones(mesh.nC)*mu_0 # magnetic permeability values for all cells
    epsilon = np.ones(mesh.nC)*epsilon_0 # dielectric constant values for all cells
    omega = 2*np.pi*f # Angular frequency (rad/s)
    sigmahat = sigma # Assume sigmahat = sigma
    # In reality ...
    #         sigmahat = sigma + 1j*epsilon*omega # sigmahat = sigma + 1j*omega*epsilon
    Div = mesh.faceDiv # Divergence matrix
    mesh.setCellGradBC([['dirichlet', 'dirichlet']]) # Setup boundary conditions
    Grad = mesh.cellGrad # Gradient matrix
    B = mesh.cellGradBC  # a matrix for boundary conditions
    Exbc = np.r_[0., 1.] # boundary values for Ex
    Msighat = Utils.sdiag(sigmahat) 
    Mmu = Utils.sdiag(mesh.aveF2CC.T * mu) 

    tempUp = sp.hstack((Grad, 1j*omega*Mmu)) # Top row of A matrix
    tempDw = sp.hstack((Msighat, Div)) # Bottom row of A matrix
    A = sp.vstack((tempUp, tempDw)) # Full A matrix
    rhs = np.r_[-B*Exbc, np.zeros(mesh.nC)] # Right-hand side   

    Ainv = Solver(A) # Factorize A matrix
    u = Ainv*rhs   # Solve A^-1 rhs = u
    Ex = u[:mesh.nC] # Extract Ex from uution vector u
    Hy = u[mesh.nC:mesh.nC+mesh.nN] # Extract Hy from solution vector u    
    P0 = sp.coo_matrix(
        (np.r_[1.], (np.r_[0], np.r_[len(u)-1])), shape=(1, len(u))
             )
    P0 = P0.tocsr()
    Zxy = - 1./(P0*u)    
    ATinv = Solver(A.T) # Factorize A matrix        
    
    if dtype == "ri":
        PTvr = (P0.T*np.r_[v[0]]).astype(complex)
        PTvi = P0.T*np.r_[v[1]]*-1j
        dZrduT_v = Utils.sdiag((1./(u**2)))*PTvr
        dZiduT_v = Utils.sdiag((1./(u**2)))*PTvi
        
        dAdsiguT = sp.hstack((Utils.spzeros(mesh.nC, mesh.nN), Utils.sdiag(Ex)))
        
        dZrdsigT_v = - (dAdsiguT*(ATinv*dZrduT_v)).real
        dZidsigT_v = - (dAdsiguT*(ATinv*dZiduT_v)).real        
        return dZrdsigT_v + dZidsigT_v
    
    elif dtype == "rhopha":
        dZadZ = Zxy / abs(Zxy)
        drhodZa = 2. * abs(Zxy) / (mu_0*omega)
        drhodZ = (drhodZa * dZadZ)
        drhodZT_v = drhodZ.conj() * np.r_[v[0]]        
        drhodsigT_v = Utils.sdiag((1./(u**2)))*(P0.T*drhodZT_v)
        dAdsiguT = sp.hstack((Utils.spzeros(mesh.nC, mesh.nN), Utils.sdiag(Ex)))
        drhodsigT_v = - (dAdsiguT*(ATinv*drhodsigT_v)).real
        
        return drhodsigT_v

In [20]:
sigma0 = sigma*3
dobs_ri = dpred(sigma, dtype="ri")
r = dpred(sigma0, dtype="ri") - dobs_ri 

Tests.checkDerivative(
    lambda m: [misfit(m, dobs=dobs_ri), Jtvec(m, r)],
    sigma0,
    plotIt=False,
    num=5
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    3.773e-04     6.383e-04      nan
 1   1.00e-02    9.181e-06     1.692e-05      1.577
 2   1.00e-03    2.419e-06     1.914e-07      1.946
 3   1.00e-04    2.591e-07     1.938e-09      1.995
 4   1.00e-05    2.608e-08     1.940e-11      1.999
========================= PASS! =========================
You deserve a pat on the back!



True

In [21]:
sigma0 = sigma*1.5
dobs_rhopha = dpred(sigma, dtype="rhopha")
r = dpred(sigma0, dtype="rhopha") - dobs_rhopha 

Tests.checkDerivative(
    lambda m: [misfit(m, dobs=dobs_rhopha, dtype="rhopha"), Jtvec(m, r, dtype="rhopha")],
    sigma0,
    plotIt=False,
    num=3, 
    dx = sigma0*2
)

==================== checkDerivative ====================
iter    h         |ft-f0|   |ft-f0-h*J0*dx|  Order
---------------------------------------------------------
 0   1.00e-01    4.068e+02     1.244e+01      nan
 1   1.00e-02    4.190e+01     2.143e-02      2.764
 2   1.00e-03    4.192e+00     2.649e-04      1.908
========================= PASS! =========================
The test be workin!



True

## Adjoint tests

In [22]:
v = np.random.rand(mesh.nC)
w = np.random.rand(dobs_ri.shape[0])
wtJv = w.dot(Jvec(sigma0, v, dtype="ri"))
vtJtw = v.dot(Jtvec(sigma0, w, dtype="ri"))
passed = np.abs(wtJv - vtJtw) < 1e-10
print('Adjoint Test', np.abs(wtJv - vtJtw), passed)

('Adjoint Test', 4.4408920985006262e-16, True)


In [23]:
v = np.random.rand(mesh.nC)
w = np.random.rand(dobs_rhopha.shape[0])
wtJv = w.dot(Jvec(sigma0, v, dtype="rhopha"))
vtJtw = v.dot(Jtvec(sigma0, w, dtype="rhopha"))
passed = np.abs(wtJv - vtJtw) < 1e-10
print('Adjoint Test', np.abs(wtJv - vtJtw), passed)

('Adjoint Test', 5.6843418860808015e-14, True)
